In [ ]:
# From chatGPT - do not use, doesn't work
# uses HybridModel and pytorch

In [6]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers
import torch
from torch import nn
from skopt import gp_minimize
from skopt.space import Integer
from skopt.utils import use_named_args

In [7]:
# Define the device
dev = qml.device('default.qubit', wires=4)

In [23]:
# Define the QVC circuit
def qvc_circuit(weights, features):
    AngleEmbedding(features, wires=range(4), rotation='Y')
    for i in range(4):
        qml.Hadamard(wires=i)
    StronglyEntanglingLayers(weights, wires=range(4))
    # Additional entangling and non-parameteric gates for expressibility
    for i in range(4):
        qml.CZ(wires=[i, (i+1) % 4])
    for i in range(4):
        qml.S(wires=i)
    # return [qml.expval(qml.PauliZ(i)) for i in range(4)]
    return qml.expval(qml.PauliZ(0))

# Define the QNode
@qml.qnode(dev, interface='torch')
def quantum_net(features, weights):
    return qvc_circuit(weights, features)

# Define the hybrid model
class HybridModel(nn.Module):
    def __init__(self, n_layers):
        super(HybridModel, self).__init__()
        self.n_layers = n_layers
        self.q_weights = nn.Parameter(0.01 * torch.randn(n_layers, 4, 3))
        self.classical_layer = nn.Linear(4, 2)

    def forward(self, x):
        q_out = quantum_net(x, self.q_weights)
        print("q_out:",q_out) 
        return self.classical_layer(q_out.float())
        #return self.classical_layer(torch.tensor(q_out, dtype=torch.float32))

# Dummy data for illustration purposes
X_train = np.random.randn(10, 4)
y_train = np.random.randint(0, 2, size=(10,))

# Define the training function
def train_model(n_layers):
    print("n_layers:",n_layers)
    model = HybridModel(n_layers)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.NAdam(model.parameters(), lr=0.01)

    # Training loop
    for epoch in range(20):  # Use a small number of epochs for illustration
        optimizer.zero_grad()
        print("X-train:",X_train)
        outputs = model(torch.tensor(X_train, dtype=torch.float32))
        loss = criterion(outputs, torch.tensor(y_train, dtype=torch.long))
        loss.backward()
        optimizer.step()

    return loss.item()

# Define the search space for Bayesian Optimization
# search_space = [Integer(1, 5, name='n_layers')]
search_space = [Integer(1, 5)]

# @use_named_args(search_space)
def objective(params):
    n_layers = params[0]
    return train_model(n_layers)

# Perform Bayesian Optimization
result = gp_minimize(objective, search_space, n_calls=10, random_state=0)

print(f"Optimal number of layers: {result.x[0]}")

n_layers: 3
X-train: [[ 1.78087531  1.72806294 -0.47268932  0.9647733 ]
 [ 1.14096773 -0.15202764 -0.89817143 -0.06870759]
 [-0.16805036 -0.24064795  0.46914166  0.22797033]
 [-1.09415865 -2.12865278 -0.30069457 -0.79925856]
 [ 1.00558691  0.49648657  0.04471134  0.96509888]
 [ 0.12206214  1.60267474 -0.60057998  0.74278274]
 [ 0.67046392 -0.41064564 -0.59932628 -0.3561062 ]
 [ 1.58123132  0.6446999   0.991755    1.63168419]
 [-0.77109517 -1.56315681  0.16853067  0.39293112]
 [ 1.84809785  0.83218425  0.62412144 -0.62200397]]
q_out: tensor([ 0.8304, -0.0535,  0.2406, -0.7061,  0.8305,  0.6873, -0.3344,  0.9970,
         0.3968, -0.5701], dtype=torch.float64, grad_fn=<MvBackward0>)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x10 and 4x2)